In [1]:
import json
import re

data = []
with open('dump_small.jsonln', 'r') as file:
    for line in file:
        data.append(json.loads(line))
        
print(f'Documents: {len(data)}')

Numero de documentos: 11225


## Functions to clean wikipedia's text.

In [2]:
def removeUrl(texto):
    # https://www.geeksforgeeks.org/python-check-url-string/
    pattern = r"""
        (?i)
        \b  
        (?:
            https?://
        |
            www
            \d{0,3}
            [.]
        |
            [a-z0-9.-]+
            [.]
            [a-z]{2,4}
            /
        )
        (?:
            [^\s()<>]+
        |
            (
            (?:
                [^\s()<>]+
            |
                (
                [^\s()<>]+
                )
            )*
            )
        )+
        (?:
            (
            (?:
                [^\s()<>]+
            |
                (
                [^\s()<>]+
                )
            )*
            )
        |
            [^\s`!()[]{};:'".,<>?«»“”‘’]
        )
    """
    repl = ''
    matcher = re.compile(pattern, re.VERBOSE)
    return matcher.sub(repl, texto)

In [3]:
def removeWikipediaLinks(text):
    pattern = r'\[\[(?:[^|]*?\|)*?([^|]*?)\]\]'
    repl = r'\1'
    matcher = re.compile(pattern)
    return matcher.sub(repl, text)

In [4]:
def removeRef(text):
    return re.sub(r'<ref(?:.*)?>.*?(?:</ref>)?', '', text)

In [5]:
def removeMath(text):
    return re.sub(r'<math(?:.*)?>.*?(?:</math>)?', '', text)

In [6]:
def removeOtherTags(text):
    return re.sub(r'<.*?>(.*?)<.*?>', r'\1', text)

In [7]:
def removeQuotationMark(text):
    pattern = r"""(['"]+)(.*?)\1"""
    matcher = re.compile(pattern)
    return matcher.sub(r'\2', text)

In [8]:
def removeEqual(text):
    return text.replace('=',' ') 

In [9]:
def removeBraces(text):
    pattern = r"{.*?}"
    matcher = re.compile(pattern)
    return matcher.sub('', text)


In [10]:
def removeBrackets(text):
    pattern = r"\[.*?\]"
    matcher = re.compile(pattern)
    text = matcher.sub('', text)
    text = text.replace(']', '')
    return text.replace('[', '')

In [11]:
def removeHtmlComment(texto):
    pattern = r"<!--.*?-->"
    matcher = re.compile(pattern)
    return matcher.sub('', texto)

In [12]:
def removeAsterisk(texto):
    texto = texto.replace("*", "")
    return texto

In [13]:
def removePx(text):     
    return re.sub(r"(?:\d+x)?\d+px", ' ', text)

removePx("o problema.624pxDurante a adolescênci")

'o problema. Durante a adolescênci'

In [14]:
def removeDash(text):
    return re.sub(r'\s+[-–]\s+', ' ', text)

removeDash("Vista - Alegre Sport Clube Beira-Mar")

'Vista Alegre Sport Clube Beira-Mar'

In [15]:
def removeParenthesis(text):
    text = text.replace('(', '')
    return text.replace(')', '')

In [16]:
def removeDigits(text):
    return re.sub('\d+', '', text)

In [17]:
def removeWikipediaTemplates(text):
    conta = 0
    spans_proibidos = []
    for item in re.finditer(r'{{|}}', text):
        if item[0] == '{{':
            if conta == 0:
                inicio = item.span()[0]
            conta += 1
        else:
            conta -= 1
            if conta == 0:
                fim = item.span()[1]
                spans_proibidos.append((inicio, fim))

    clean_text = ''
    inicio = 0
    for span in spans_proibidos:
        fim, novo_inicio = span
        clean_text += text[inicio:fim]
        inicio = novo_inicio
    
    clean_text += text[inicio:]
    return clean_text



In [18]:
def clearText(text):
    funcs = [
        removeUrl,
        removeWikipediaLinks,
        removeRef,
        removeMath,
        removeOtherTags,
        removeQuotationMark,
        removeEqual,
        removeBraces,
        removeBrackets,
        removeHtmlComment,
        removeAsterisk,
        removePx,
        removeDash,
        removeParenthesis,
        removeDigits,
        removeWikipediaTemplates
    ]
    x = text
    for func in funcs:
        x = func(x)
    return x

In [19]:
import json
with open('dump_small_clean.jsonln', 'w', encoding="utf8") as file:
    for i in range(len(data)):
        json.dump({"body": clearText(data[i]['body']), "title": data[i]["title"]}, file)
        file.write('\n')